In [331]:
import requests
from bs4 import BeautifulSoup
import re
import json
import PIL
from PIL import ImageDraw
from PIL import ImageFont
import os
import slugify
from tqdm import tqdm

In [332]:
# Define the URL of the web page
url = 'https://hayday.fandom.com/wiki/Goods_List'

# Use the requests library to fetch the web page content
response = requests.get(url)

# Check the status of the request
# A status code of 200 means the request was successful
if response.status_code == 200:
    # Get the content of the response
    page_content = response.text
else:
    print("Failed to fetch the web page. Status code: ", response.status_code)

In [333]:
def convert_to_seconds(time_str):
    if time_str.startswith('Instant'):
        return 0
    # Splitting the string by spaces to extract numbers and units
    parts = time_str.split()

    # Initializing total seconds
    total_seconds = 0

    # Iterating through each part of the time string
    for i in range(0, len(parts), 2):
        number = int(parts[i])
        unit = parts[i+1]

        if unit == 'h':
            # Convert hours to seconds
            total_seconds += number * 3600
        elif unit == 'min':
            # Convert minutes to seconds
            total_seconds += number * 60
        elif unit == 'd':
            # Convert days to seconds
            total_seconds += number * 86400

    return total_seconds

def seconds_to_time_str(seconds):
    # Convert seconds to days, hours, minutes
    days = seconds // 86400
    hours = (seconds % 86400) // 3600
    minutes = (seconds % 3600) // 60

    # Create a list of strings to store the parts of the time string
    time_parts = []

    if days > 0:
        time_parts.append(str(days))
        time_parts.append('d')

    if hours > 0:
        time_parts.append(str(hours))
        time_parts.append('h')

    if minutes > 0:
        time_parts.append(str(minutes))
        time_parts.append('min')

    # Join the time parts to create the time string
    return ' '.join(time_parts)

In [334]:
soup = BeautifulSoup(page_content, 'html.parser')

tables = soup.find_all('table')

# Get the table with the goods
goods_table = tables[0]

# Get the rows of the table
goods_rows = goods_table.find_all('tr')

# Get the header row
header_row = goods_rows[0]

# Get the column names
header_row_data = header_row.find_all('th')

# Create a list of column names
column_names = []
for header in header_row_data:
    column_names.append(header.text.strip())

# Create a list of rows
rows = []
for row in goods_rows[1:]:
    row_data = row.find_all('td')
    row_dict = {}
    for i, cell in enumerate(row_data):
        cell_text = cell.text.strip()
        if i == 0:
            cell_text = cell.find('a').text.strip()
            # Add the image to the "image" column
            image = cell.find('img')
            if image:
                if 'data-src' in image.attrs:
                    row_dict['image'] = image['data-src']
                else:
                    row_dict['image'] = image['src']
        if i == 3:
            cell_text = cell_text.split('★★★')[0].strip()
            cell_text = convert_to_seconds(cell_text)
        if i==5:
            
            matches = re.finditer(r'([\w\s]+)\s\((\d+)\)', cell_text, re.MULTILINE)
            needs=[] # {name: amount} 
            for matchNum, match in enumerate(matches, start=1):
                needs.append({match.group(1).strip(): int(match.group(2).strip())})
            row_dict[column_names[i]] = needs
            continue
        if i==6:
            a = cell.find('a')
            cell_text = a.text.strip()
        row_dict[column_names[i]] = cell_text
    rows.append(row_dict)

# # download all images to folder images and add path to json
# # image name is the name of the good
# os.makedirs('images', exist_ok=True)
# for row in tqdm(rows):
#     if 'image' in row:
#         image_url = row['image']
#         image_name = row['Name']
#         image_name = image_name.replace(' ', '_')
#         image_name = slugify.slugify(image_name)
#         image_name = image_name.lower()
#         image_name = image_name + '.png'
#         image_path = os.path.join('images', image_name)
#         # Download the image
#         response = requests.get(image_url)
#         if response.status_code == 200:
#             with open(image_path, 'wb') as f:
#                 f.write(response.content)
#             row['image'] = image_path
#         else:
#             print("Failed to download image. Status code: ", response.status_code)

# with open('goods.json', 'w') as f:
#     json.dump(rows, f, indent=2)

In [335]:
with open('goods.json', 'r') as f:
    rows = json.load(f)

In [336]:
def find_by_name(name):
    for row in rows:
        if row['Name'].lower() == name.lower():
            return row
    return None

def find_needed_for(name,count):
    row = find_by_name(name)
    if row is None:
        raise Exception(f'Could not find {name}')
    result = row.copy()
    result['needs'] = []
    result['count'] = count
    for need in row['Needs']:
        for key, value in need.items():
            if (key==name):
                continue
            result['needs'].append(find_needed_for(key, value*count))
    del result['Needs']
    return result


In [337]:
from graphviz import Digraph
import random


def draw_graph(node, graph):
    if 'needs' in node:
        for need in node['needs']:
            graph.edge(need['Name'], node['Name'])
            draw_graph(need, graph)
    graph.node(node['Name'], label=f"{node['Name']}",
                labelloc='b',
                image='/Users/attorneyking/Desktop/js/'+node['image'])

def draw_recipe_needed(recipe_tree):
    graph = Digraph(comment='Hayday Goods item', format='png',strict=True)
    graph.attr(label=f'Recipe for {recipe_tree["Name"]}')
    graph.attr(size='100,100')
    graph.attr('node', shape='none')
    draw_graph(recipe_tree, graph)
    graph.render(f'recipes/{recipe_tree["Name"]}')

In [338]:
# random_row = random.choice(rows)
for random_row in tqdm(rows):
    try:
        test = find_needed_for(random_row['Name'], 1)
        draw_recipe_needed(test)
    except Exception as e:
        print('Failed to draw recipe for', random_row['Name'])
        print(e)
        continue

 28%|██▊       | 107/380 [00:17<00:33,  8.15it/s]

Failed to draw recipe for Diamond ring
Could not find Diamond


 55%|█████▌    | 209/380 [00:37<00:21,  7.81it/s]

Failed to draw recipe for Gracious bouquet
Could not find Diamond


100%|██████████| 380/380 [01:07<00:00,  5.61it/s]
